In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Create minimal model input

- Takes the flights data
- Processes the schedule/realized datetimes and computes the delay in seconds
- Remove observations with unknown prediction targets
- Write prediction target with minimal feature set to CSV

### Parameters

-------------------
- `input_file`: Filepath of flights data in format received from Schiphol
- `output_file`: Filepath to write output csv file with minimal modelling input


### Returns

-----------------

Output CSV file written to `output_file` with minimal model input

Example output,
    
    id                   |  aircraftRegistration  |  airlineCode  |  terminal  |  serviceType  |      scheduleDateTime        |     actualOffBlockTime      |  scheduleDelaySeconds
    124257473326719795   |    PHEXI               |     80.0      |     2.0    |       J       |  2018-05-01 16:35:00+02:00   |  2018-05-01 16:58:16+02:00  |         1396.0
    124538476600837715   |    PHEXL               |     2481.0    |     1.0    |       J       |  2018-06-10 13:00:00+02:00   |  2018-06-10 13:11:25+02:00  |         685.0
    123512829091050355   |    PHBQO               |     100.0     |     2.0    |       J       |  2018-01-15 10:15:00+01:00   |  2018-01-15 10:35:10+01:00  |         1210.0
    123786805997701057   |    PHEXG               |     2481.0    |     1.0    |       J       |  2018-02-23 17:45:00+01:00   |  2018-02-23 17:55:52+01:00  |         652.0
    124664922607744671   |    PHBXP               |     1551.0    |     2.0    |       J       |  2018-06-28 20:50:00+02:00   |  2018-06-28 22:09:23+02:00  |         4763.0


# File parameters

In [3]:
# input parameters cell
input_file = "../lvt-schiphol-assignment-snakemake/data/raw/flights.csv"
output_file = "processed_flights.csv"

## Imports

In [4]:
import pandas as pd
import numpy as np

import sys
sys.path.append("../")

from src.data.google_storage_io import read_csv_data, write_csv_data

# Utility functions

In [5]:
def missing_values_percentages(df):
    """Calculate summary of missing values per column"""
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing})

    missing_value_df = missing_value_df.sort_values('percent_missing', ascending=False)
    return missing_value_df


def check_col_singular(x):
    """check if pd.Series contains more than 1 unique value excluding NaN"""
    return x.dropna().nunique() <= 1


def drop_singular_columns(df, verbose=False):
    """Drop DataFrame columns with 1 or fewer unique values excluding NaN"""
    col_singular = df.apply(check_col_singular, axis=0)
    if verbose:
        n_singular = sum(col_singular)
        print(f"Dropping {n_singular} columns")
        print(f"{col_singular[col_singular].index}")
        
    df_output = df[[col for col, is_singular in col_singular.items() 
                    if not is_singular]]
    return df_output


def clean_flights(df_flights, verbose=True):
    """Clean flights data by removing singular columns and formatting dates"""
    df = df_flights
    df = df.dropna(subset=["scheduleDate", "scheduleTime", "actualOffBlockTime"]).reset_index()
    
    # remove singular columns
    df = drop_singular_columns(df, verbose=verbose)
    
    # format datetime fields
    df["actualOffBlockTime"] = pd.to_datetime(df["actualOffBlockTime"], utc=True).dt.tz_convert('Europe/Amsterdam')
    
    series_datetime_str = df['scheduleDate'].astype(str) + " " + df['scheduleTime'].astype(str)
    df["scheduleDateTime"] = pd.to_datetime(series_datetime_str, format="%Y%m%d %H:%M:%S").dt.tz_localize('Europe/Amsterdam')
    
#     calculate delay as difference between scheduled and realized departure
    df["scheduleDelaySeconds"] = pd.to_timedelta(df["actualOffBlockTime"] - df["scheduleDateTime"]).dt.total_seconds()

    return df


def read_flights_data(filename):
    """Read data local or from Google Storage bucket and clean it"""
    df = read_csv_data(input_file)
    print(f"Loaded data from: {input_file}\n"
          f"Shape of data: {df.shape}")
    
    df = clean_flights(df)
    print(f"Cleaned flights data\n"
          f"Shape of data: {df.shape}")
    
    return df

## Read data

In [6]:
%%time
df = read_csv_data(input_file)

Reading file from local directory
File:	../lvt-schiphol-assignment-snakemake/data/raw/flights.csv

Wall time: 1.84 s


In [8]:
df.shape, df["id"].drop_duplicates().shape

((523275, 28), (512204,))

In [24]:
df[df["id"].isin(df["id"][df[["id"]].duplicated()].unique())].shape, df[df["id"].isin(df["id"][df[["id"]].duplicated()].unique())].drop_duplicates().shape

((22106, 28), (11035, 28))

In [32]:
df2 = df[df["id"].isin(df["id"][df[["id"]].duplicated()].unique())].drop_duplicates()
df2[["id"]].duplicated().sum()

0

In [26]:
df[df["id"].isin(df["id"][df[["id"]].duplicated()].unique())].sort_values("id").drop_duplicates().to_csv("dupl.ipynb_checkpoints/cates.csv")

# Output prediction target.ipynb_checkpoints/

In [7]:
df.columns

Index(['index', 'actualOffBlockTime', 'aircraftRegistration',
       'aircraftType.iatamain', 'aircraftType.iatasub', 'airlineCode',
       'expectedTimeBoarding', 'expectedTimeGateClosing',
       'expectedTimeGateOpen', 'flightName', 'flightNumber', 'gate', 'id',
       'mainFlight', 'prefixIATA', 'prefixICAO', 'publicEstimatedOffBlockTime',
       'publicFlightState.flightStates', 'route.destinations', 'scheduleDate',
       'scheduleTime', 'serviceType', 'terminal',
       'transferPositions.transferPositions', 'scheduleDateTime',
       'scheduleDelaySeconds'],
      dtype='object')

In [8]:
# meta columns for utility for columns we will often merge by
output_columns = ["id", "aircraftRegistration", "airlineCode", "terminal", 
                  "serviceType", "scheduleDateTime", "actualOffBlockTime", "scheduleDelaySeconds"]

# DataFrame with id + merging columns + prediction target
df_target = df[output_columns]
df_target.head()

,id,aircraftRegistration,airlineCode,terminal,serviceType,scheduleDateTime,actualOffBlockTime,scheduleDelaySeconds
0,123414481790510775,PHPXB,148.0,NaN,NaN,2018-01-01 03:30:00+01:00,2018-01-01 03:22:00+01:00,-480.0
1,123414479288269149,PHHSJ,164.0,1.0,J,2018-01-01 06:00:00+01:00,2018-01-01 05:58:22+01:00,-98.0
2,123414479666542945,PHHSG,100.0,1.0,J,2018-01-01 06:05:00+01:00,2018-01-01 06:00:00+01:00,-300.0
3,123414479288365061,PHHSG,164.0,1.0,J,2018-01-01 06:05:00+01:00,2018-01-01 06:00:00+01:00,-300.0
4,123414479288274329,PHHXB,164.0,1.0,J,2018-01-01 06:15:00+01:00,2018-01-01 06:26:34+01:00,694.0


## Write output to CSV

Local or Google Storage is both handled

In [11]:
# write output file
write_csv_data(df_target, output_file, index=False)

Writing file to local directory
File:	processed_flights.csv



### Overview of the output data

In [13]:
df_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487716 entries, 0 to 487715
Data columns (total 8 columns):
id                      487716 non-null int64
aircraftRegistration    487713 non-null object
airlineCode             486503 non-null float64
terminal                477391 non-null float64
serviceType             482937 non-null object
scheduleDateTime        487716 non-null datetime64[ns, Europe/Amsterdam]
actualOffBlockTime      487716 non-null datetime64[ns, Europe/Amsterdam]
scheduleDelaySeconds    487716 non-null float64
dtypes: datetime64[ns, Europe/Amsterdam](2), float64(3), int64(1), object(2)
memory usage: 29.8+ MB
